In [ ]:
#Defining the model

model = social_stgcnn(n_stgcnn =args.n_stgcnn,n_txpcnn=args.n_txpcnn,
output_feat=args.output_size,seq_len=args.obs_seq_len,
kernel_size=args.kernel_size,pred_seq_len=args.pred_seq_len)


#Training settings

optimizer = optim.SGD(model.parameters(),lr=args.lr)

if args.use_lrschd:
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_sh_rate, gamma=0.1)



checkpoint_dir = './checkpoint/'+args.tag+'/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

with open(checkpoint_dir+'args.pkl', 'wb') as fp:
    pickle.dump(args, fp)



print('Data and model loaded')
print('Checkpoint dir:', checkpoint_dir)

#Training
metrics = {'train_loss':[],  'val_loss':[]}
constant_metrics = {'min_val_epoch':-1, 'min_val_loss':9999999999999999}

def train(epoch):
    global metrics,loader_train
    model.train()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_train)
    turn_point =int(loader_len/args.batch_size)*args.batch_size+ loader_len%args.batch_size -1


    for cnt,batch in enumerate(loader_train):
        batch_count+=1

        #Get data
        batch = [tensor for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
         loss_mask,V_obs,A_obs,A_dir_obs, V_tr,A_tr,A_dir_tr = batch


        optimizer.zero_grad()
        #Forward
        #V_obs = batch,seq,node,feat
        #V_obs_tmp = batch,feat,seq,node
        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_,_ = model(V_obs_tmp,A_obs.squeeze(), A_dir_obs.squeeze())

        V_pred = V_pred.permute(0,2,3,1)



        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        A_dir_tr = A_dir_tr.squeeze()
        V_pred = V_pred.squeeze()

        if batch_count%args.batch_size !=0 and cnt != turn_point :
            l = graph_loss(V_pred,V_tr)
            if is_fst_loss :
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss/args.batch_size
            is_fst_loss = True
            loss.backward()

            if args.clip_grad is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(),args.clip_grad)


            optimizer.step()
            #Metrics
            loss_batch += loss.item()
            print('TRAIN:','\t Epoch:', epoch,'\t Loss:',loss_batch/batch_count)

    metrics['train_loss'].append(loss_batch/batch_count)




def vald(epoch):
    global metrics,loader_val,constant_metrics
    model.eval()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_val)
    turn_point =int(loader_len/args.batch_size)*args.batch_size+ loader_len%args.batch_size -1

    for cnt,batch in enumerate(loader_val):
        batch_count+=1

        #Get data
        batch = [tensor for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
         loss_mask,V_obs,A_obs,A_dir_obs, V_tr,A_tr,A_dir_tr = batch


        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_,_ = model(V_obs_tmp,A_obs.squeeze(), A_dir_obs.squeeze())

        V_pred = V_pred.permute(0,2,3,1)

        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        A_dir_tr = A_dir_tr.squeeze()
        V_pred = V_pred.squeeze()

        if batch_count%args.batch_size !=0 and cnt != turn_point :
            l = graph_loss(V_pred,V_tr)
            if is_fst_loss :
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss/args.batch_size
            is_fst_loss = True
            #Metrics
            loss_batch += loss.item()
            print('VALD:','\t Epoch:', epoch,'\t Loss:',loss_batch/batch_count)

    metrics['val_loss'].append(loss_batch/batch_count)

    if  metrics['val_loss'][-1]< constant_metrics['min_val_loss']:
        constant_metrics['min_val_loss'] =  metrics['val_loss'][-1]
        constant_metrics['min_val_epoch'] = epoch
        torch.save(model.state_dict(),checkpoint_dir+'val_best.pth')  # OK
        check_test_performance()

print('Training started ...')
for epoch in range(args.num_epochs):
    train(epoch)
    vald(epoch)
    if args.use_lrschd:
        scheduler.step()


    print('*'*30)
    print('Epoch:',args.tag,":", epoch)
    for k,v in metrics.items():
        if len(v)>0:
            print(k,v[-1])


    print(constant_metrics)
    print('*'*30)

    with open(checkpoint_dir+'metrics.pkl', 'wb') as fp:
        pickle.dump(metrics, fp)

    with open(checkpoint_dir+'constant_metrics.pkl', 'wb') as fp:
        pickle.dump(constant_metrics, fp)



Data and model loaded
Checkpoint dir: ./checkpoint/social-tag/
Training started ...
TRAIN: 	 Epoch: 0 	 Loss: 0.016137098893523216
TRAIN: 	 Epoch: 0 	 Loss: 0.015971467830240726
TRAIN: 	 Epoch: 0 	 Loss: 0.01582263099650542
TRAIN: 	 Epoch: 0 	 Loss: 0.01569222123362124
TRAIN: 	 Epoch: 0 	 Loss: 0.01556869801133871
TRAIN: 	 Epoch: 0 	 Loss: 0.015435969922691584
TRAIN: 	 Epoch: 0 	 Loss: 0.015314615198544093
TRAIN: 	 Epoch: 0 	 Loss: 0.015195480664260685
TRAIN: 	 Epoch: 0 	 Loss: 0.015073743441866504
TRAIN: 	 Epoch: 0 	 Loss: 0.014958398323506117
TRAIN: 	 Epoch: 0 	 Loss: 0.014832561158321121
TRAIN: 	 Epoch: 0 	 Loss: 0.014712462977816662
TRAIN: 	 Epoch: 0 	 Loss: 0.014586966413144883
TRAIN: 	 Epoch: 0 	 Loss: 0.014463134109973907
TRAIN: 	 Epoch: 0 	 Loss: 0.014330958699186642
TRAIN: 	 Epoch: 0 	 Loss: 0.014194586256053299
TRAIN: 	 Epoch: 0 	 Loss: 0.014058986569152158
TRAIN: 	 Epoch: 0 	 Loss: 0.013917815768056445
TRAIN: 	 Epoch: 0 	 Loss: 0.013773563170903608
TRAIN: 	 Epoch: 0 	 Loss: 

100%|██████████| 70/70 [00:02<00:00, 25.85it/s]

Testing ....


ADE: 1.9133237653693322  FDE: 2.7102087273278452
**************************************************
******************************
Epoch: social-tag : 0
train_loss 0.013322172451704152
val_loss 0.012009402612845103
{'min_val_epoch': 0, 'min_val_loss': 0.012009402612845103}
******************************
TRAIN: 	 Epoch: 1 	 Loss: 0.009136193431913853
TRAIN: 	 Epoch: 1 	 Loss: 0.00898823793977499
TRAIN: 	 Epoch: 1 	 Loss: 0.008789907830456892
TRAIN: 	 Epoch: 1 	 Loss: 0.008572125108912587
TRAIN: 	 Epoch: 1 	 Loss: 0.008344466891139745
TRAIN: 	 Epoch: 1 	 Loss: 0.008179783743495742
TRAIN: 	 Epoch: 1 	 Loss: 0.008020659309944935
TRAIN: 	 Epoch: 1 	 Loss: 0.007809473667293787
TRAIN: 	 Epoch: 1 	 Loss: 0.007620842920409309
TRAIN: 	 Epoch: 1 	 Loss: 0.007457277784124017
TRAIN: 	 Epoch: 1 	 Loss: 0.007289383348754861
TRAIN: 	 Epoch: 1 	 Loss: 0.007147022445375721
TRAIN: 	 Epoch: 1 	 Loss: 0.0070416820951952385
TRAIN: 	 Epoch: 1 	 Loss: 0.007041029365999358
TRAIN: 	 Epoch: 1 	 Loss: 0.007008424

100%|██████████| 70/70 [00:02<00:00, 25.54it/s]

...

Testing ....


ADE: 0.9093217921154171  FDE: 1.4357146606614044
**************************************************
******************************
Epoch: social-tag : 115
train_loss -0.012440763940931009
val_loss -0.007709600539370017
{'min_val_epoch': 115, 'min_val_loss': -0.007709600539370017}
******************************
TRAIN: 	 Epoch: 116 	 Loss: -0.012574545107781887
TRAIN: 	 Epoch: 116 	 Loss: -0.012848238460719585
TRAIN: 	 Epoch: 116 	 Loss: -0.013043818684915701
TRAIN: 	 Epoch: 116 	 Loss: -0.012452329508960247
TRAIN: 	 Epoch: 116 	 Loss: -0.011914950050413608
TRAIN: 	 Epoch: 116 	 Loss: -0.012021400034427643
TRAIN: 	 Epoch: 116 	 Loss: -0.012176100430744035
TRAIN: 	 Epoch: 116 	 Loss: -0.012259699054993689
TRAIN: 	 Epoch: 116 	 Loss: -0.012262418452236388
TRAIN: 	 Epoch: 116 	 Loss: -0.012093075923621655
TRAIN: 	 Epoch: 116 	 Loss: -0.011916487477719784
TRAIN: 	 Epoch: 116 	 Loss: -0.011891651432961226
TRAIN: 	 Epoch: 116 	 Loss: -0.011934426326591235
TRAIN: 	 Epoch: 116 	 Loss: -0.0120495

100%|██████████| 70/70 [00:02<00:00, 26.10it/s]

Testing ....


ADE: 0.7251130762181001  FDE: 1.1926335161153436
**************************************************
******************************
Epoch: social-tag : 140
train_loss -0.012745812508533415
val_loss -0.008992135016755625
{'min_val_epoch': 140, 'min_val_loss': -0.008992135016755625}
******************************
TRAIN: 	 Epoch: 141 	 Loss: -0.013307278044521809
TRAIN: 	 Epoch: 141 	 Loss: -0.013414918445050716
TRAIN: 	 Epoch: 141 	 Loss: -0.013692566193640232
TRAIN: 	 Epoch: 141 	 Loss: -0.013802569825202227
TRAIN: 	 Epoch: 141 	 Loss: -0.013780885562300681
TRAIN: 	 Epoch: 141 	 Loss: -0.013763686486830315
TRAIN: 	 Epoch: 141 	 Loss: -0.013616751613361495
TRAIN: 	 Epoch: 141 	 Loss: -0.013602201594039798
TRAIN: 	 Epoch: 141 	 Loss: -0.013527108045915762
TRAIN: 	 Epoch: 141 	 Loss: -0.01330742621794343
TRAIN: 	 Epoch: 141 	 Loss: -0.01327093131840229
TRAIN: 	 Epoch: 141 	 Loss: -0.01336667329693834
TRAIN: 	 Epoch: 141 	 Loss: -0.013344743217413243
TRAIN: 	 Epoch: 141 	 Loss: -0.0132279437

In [ ]:
x = 2 + 2

# Testowanie

Jak działa poniższy kod?

**Funkcja testowania:** Zdefiniowana funkcja test(KSTEPS=20) przeprowadza walidację modelu. Używa ona załadowanego modelu do przewidywania trajektorii i obliczania metryk ADE (Average Displacement Error) i FDE (Final Displacement Error) dla wielu próbek trajektorii.

**Przygotowanie danych:** Dane testowe są ładowane za pomocą TrajectoryDataset i DataLoader. Dane te są następnie przekazywane do modelu w partii.

**Przewidywanie trajektorii:** Model wykonuje przewidywanie trajektorii (V_pred) na podstawie obserwowanych trajektorii (V_obs). Przewidywania są reprezentowane jako rozkłady wielowymiarowe z parametrami określonymi przez model.

**Próbkowanie i obliczanie metryk:** Dla każdej próbki trajektorii, skrypt próbkuje KSTEPS przewidywanych trajektorii z modelu, a następnie oblicza ADE i FDE dla każdej próbki. Wyniki są zapisywane w słowniku raw_data_dict.

**Obliczanie średnich metryk:** Po przetestowaniu wszystkich partii danych, skrypt oblicza średni ADE i FDE dla wszystkich obiektów.

**Iteracja po eksperymentach:** Skrypt iteruje przez różne zapisane modele (znajdujące się w katalogu ./checkpoint/), ładuje każdy z nich i przeprowadza testy, zbierając wyniki ADE i FDE.
Wyświetlanie wyników: Na koniec skrypt wyświetla średni ADE i FDE dla wszystkich przetestowanych modeli.


In [ ]:
import os
import math
import sys
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pickle
import argparse
import glob
import torch.distributions.multivariate_normal as torchdist
import copy

def test(model,loader_test, KSTEPS=20):
    model.eval()
    ade_bigls = []
    fde_bigls = []
    raw_data_dict = {}
    step =0
    for batch in loader_test:
        step+=1
        #Get data
        batch = [tensor for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
         loss_mask,V_obs,A_obs,A_dir_obs,V_tr,A_tr,A_dir_tr = batch


        num_of_objs = obs_traj_rel.shape[1]

        #Forward
        #V_obs = batch,seq,node,feat
        #V_obs_tmp = batch,feat,seq,node
        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_,_ = model(V_obs_tmp,A_obs.squeeze(),A_dir_obs.squeeze())
        # print(V_pred.shape)
        # torch.Size([1, 5, 12, 2])
        # torch.Size([12, 2, 5])
        V_pred = V_pred.permute(0,2,3,1)
        # torch.Size([1, 12, 2, 5])>>seq,node,feat
        # V_pred= torch.rand_like(V_tr).cuda()


        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        A_dir_tr = A_dir_tr.squeeze()
        V_pred = V_pred.squeeze()
        num_of_objs = obs_traj_rel.shape[1]
        V_pred,V_tr =  V_pred[:,:num_of_objs,:],V_tr[:,:num_of_objs,:]
        #print(V_pred.shape)

        #For now I have my bi-variate parameters
        #normx =  V_pred[:,:,0:1]
        #normy =  V_pred[:,:,1:2]
        sx = torch.exp(V_pred[:,:,2]) #sx
        sy = torch.exp(V_pred[:,:,3]) #sy
        corr = torch.tanh(V_pred[:,:,4]) #corr

        cov = torch.zeros(V_pred.shape[0],V_pred.shape[1],2,2)
        cov[:,:,0,0]= sx*sx
        cov[:,:,0,1]= corr*sx*sy
        cov[:,:,1,0]= corr*sx*sy
        cov[:,:,1,1]= sy*sy
        mean = V_pred[:,:,0:2]

        mvnormal = torchdist.MultivariateNormal(mean,cov)


        ### Rel to abs
        ##obs_traj.shape = torch.Size([1, 6, 2, 8]) Batch, Ped ID, x|y, Seq Len

        #Now sample 20 samples
        ade_ls = {}
        fde_ls = {}
        V_x = seq_to_nodes(obs_traj.data.cpu().numpy().copy())
        V_x_rel_to_abs = nodes_rel_to_nodes_abs(V_obs.data.cpu().numpy().squeeze().copy(),
                                                 V_x[0,:,:].copy())

        V_y = seq_to_nodes(pred_traj_gt.data.cpu().numpy().copy())
        V_y_rel_to_abs = nodes_rel_to_nodes_abs(V_tr.data.cpu().numpy().squeeze().copy(),
                                                 V_x[-1,:,:].copy())

        raw_data_dict[step] = {}
        raw_data_dict[step]['obs'] = copy.deepcopy(V_x_rel_to_abs)
        raw_data_dict[step]['trgt'] = copy.deepcopy(V_y_rel_to_abs)
        raw_data_dict[step]['pred'] = []

        for n in range(num_of_objs):
            ade_ls[n]=[]
            fde_ls[n]=[]

        for k in range(KSTEPS):

            V_pred = mvnormal.sample()



            #V_pred = seq_to_nodes(pred_traj_gt.data.numpy().copy())
            V_pred_rel_to_abs = nodes_rel_to_nodes_abs(V_pred.data.cpu().numpy().squeeze().copy(),
                                                     V_x[-1,:,:].copy())
            raw_data_dict[step]['pred'].append(copy.deepcopy(V_pred_rel_to_abs))

           # print(V_pred_rel_to_abs.shape) #(12, 3, 2) = seq, ped, location
            for n in range(num_of_objs):
                pred = []
                target = []
                obsrvs = []
                number_of = []
                pred.append(V_pred_rel_to_abs[:,n:n+1,:])
                target.append(V_y_rel_to_abs[:,n:n+1,:])
                obsrvs.append(V_x_rel_to_abs[:,n:n+1,:])
                number_of.append(1)

                ade_ls[n].append(ade(pred,target,number_of))
                fde_ls[n].append(fde(pred,target,number_of))

        for n in range(num_of_objs):
            ade_bigls.append(min(ade_ls[n]))
            fde_bigls.append(min(fde_ls[n]))

    ade_ = sum(ade_bigls)/len(ade_bigls)
    fde_ = sum(fde_bigls)/len(fde_bigls)
    return ade_,fde_,raw_data_dict

#check how our new best saved model performs on test dataset
######################################################################################################
paths = ['/kaggle/working/checkpoint/social-tag']
KSTEPS=20

print("*"*50)
print('Number of samples:',KSTEPS)
print("*"*50)

for feta in range(len(paths)):
    ade_ls = []
    fde_ls = []
    path = paths[feta]
    exps = glob.glob(path)
    print('Model being tested are:',exps)

    for exp_path in exps:
        print("*"*50)
        print("Evaluating model:",exp_path)

        model_path = exp_path+'/val_best.pth'
        args_path = exp_path+'/args.pkl'
        with open(args_path,'rb') as f:
            args = pickle.load(f)

        #Data prep
        obs_seq_len = args.obs_seq_len
        pred_seq_len = args.pred_seq_len
        data_set = '/kaggle/input/dataset/datasets/'+args.dataset+'/'

        dset_test = TrajectoryDataset(
                data_set+'test/',
                obs_len=obs_seq_len,
                pred_len=pred_seq_len,
                skip=1,norm_lap_matr=True)

        loader_test = DataLoader(
                dset_test,
                batch_size=1,#This is irrelative to the args batch size parameter
                shuffle =False,
                num_workers=1)



        #Defining the model
        modelka = social_stgcnn(n_stgcnn =args.n_stgcnn,n_txpcnn=args.n_txpcnn,
        output_feat=args.output_size,seq_len=args.obs_seq_len,
        kernel_size=args.kernel_size,pred_seq_len=args.pred_seq_len)
        modelka.load_state_dict(torch.load(model_path))


        ade_ =999999
        fde_ =999999
        print("Testing ....")
        ad,fd,raw_data_dic_= test(modelka,loader_test)
        ade_= min(ade_,ad)
        fde_ =min(fde_,fd)
        ade_ls.append(ade_)
        fde_ls.append(fde_)
        print("ADE:",ade_," FDE:",fde_)




    print("*"*50)
############################################################################################################################

**************************************************
Number of samples: 20
**************************************************
Model being tested are: ['/kaggle/working/checkpoint/social-tag']
**************************************************
Evaluating model: /kaggle/working/checkpoint/social-tag
Processing Data .....


100%|██████████| 70/70 [00:02<00:00, 25.95it/s]

Testing ....


ADE: 0.7154673519729199  FDE: 1.1690865252039901
**************************************************
